<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html week14.do.txt --no_mako -->
<!-- dom:TITLE: Advanced machine learning and data analysis for the physical sciences -->

# Advanced machine learning and data analysis for the physical sciences
**Morten Hjorth-Jensen**, Department of Physics and Center for Computing in Science Education, University of Oslo, Norway and Department of Physics and Astronomy and Facility for Rare Isotope Beams, Michigan State University, East Lansing, Michigan, USA

Date: **April 23, 2024**

## Plans for the week April 22-26, 2024

**Deep generative models.**

1. Variational Autoencoders (VAE)

2. GANs

3. Reading recommendation: Goodfellow et al chapter 20.10-20-14
<!-- o [Video of lecture](https://youtu.be/0VBmdP_iCzA) -->
<!-- o [Whiteboard notes](https://github.com/CompPhysics/AdvancedMachineLearning/blob/main/doc/HandwrittenNotes/2024/NotesApril16.pdf) -->